# Streaming FLIM Data Example

This notebook demonstrates how to use the FLIM Processing Library to receive and process streaming FLIM data via UDP from TCSPC (Time-Correlated Single Photon Counting) hardware.

## Overview

The library supports real-time FLIM data processing through:
- UDP stream reception from acquisition hardware
- Automatic message parsing and data extraction
- Memory-mapped file reading for efficient data transfer
- Sequential frame processing with snapshot support

This example shows how to set up a receiver, process incoming data, and visualize results in real-time.

## Setup

Import the necessary libraries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from flim_processing import (
    FlimParams,
    StreamReceiver,
    DataManager,
    PhasorComputer,
    FittingEngine,
    SeriesMetadata,
    ElementData,
    EndSeries
)
import time

# Set up matplotlib for inline plotting
%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 5)

## Simulating a Data Stream

For this example, we'll create a simple data sender to simulate TCSPC hardware. In a real scenario, this would be your acquisition system.

**Note**: The actual UDP streaming requires a separate sender process. This notebook demonstrates the receiver side.

In [ ]:
# Configuration for streaming
UDP_PORT = 4444
UDP_ADDR = "127.0.0.1"

# FLIM parameters
period = 12.5  # nanoseconds (80 MHz laser)
time_bins = 256

flim_params = FlimParams(
    period=period,
    fit_start=10,
    fit_end=200
)

print(f"Configured to receive on {UDP_ADDR}:{UDP_PORT}")
print(f"FLIM parameters: period={period} ns, fit_range=[{flim_params.fit_start}, {flim_params.fit_end})")

## Setting Up the Stream Receiver

The `StreamReceiver` class handles UDP communication and message parsing:

In [ ]:
# Create a stream receiver
receiver = StreamReceiver(port=UDP_PORT, addr=UDP_ADDR)

print(f"StreamReceiver created and ready to receive data")
print(f"Listening on port {UDP_PORT}")

## Processing Streaming Data

The receiver provides a generator that yields events as data arrives:
- `SeriesMetadata`: When a new series starts
- `ElementData`: When a data frame arrives
- `EndSeries`: When a series ends

**Important**: This cell will block until data is received or you interrupt it. Make sure your data sender is running before executing this cell.

In [ ]:
# Initialize variables for data processing
data_manager = None
frame_count = 0
max_frames = 10  # Process up to 10 frames for this example

# Storage for results
phasor_results = []
lifetime_results = []

print("Starting to receive data...")
print("(Press Ctrl+C to stop)\n")

try:
    for event in receiver.start_receiving():
        if isinstance(event, SeriesMetadata):
            # New series started
            print(f"\n=== New Series {event.series_no} ===")
            print(f"Shape: {event.shape}")
            print(f"Data type: {event.dtype}")
            
            # Initialize data manager for this series
            data_manager = DataManager(
                shape=event.shape,
                dtype=event.dtype,
                delta_mode=False
            )
            frame_count = 0
            
        elif isinstance(event, ElementData):
            # New frame received
            frame_count += 1
            print(f"Frame {frame_count}: seqno={event.seqno}, shape={event.frame.shape}")
            
            # Add to data manager
            data_manager.add_element(event.seqno, event.frame)
            
            # Process the frame
            photon_count = event.frame
            
            # Compute phasor
            phasor = PhasorComputer.compute_phasor(photon_count, flim_params)
            phasor_results.append(phasor)
            
            # Compute lifetime (RLD)
            rld_result = FittingEngine.compute_rld(photon_count, flim_params)
            lifetime_results.append(rld_result.tau)
            
            print(f"  Phasor: g=[{np.nanmin(phasor[..., 0]):.3f}, {np.nanmax(phasor[..., 0]):.3f}], "
                  f"s=[{np.nanmin(phasor[..., 1]):.3f}, {np.nanmax(phasor[..., 1]):.3f}]")
            print(f"  Lifetime: [{np.nanmin(rld_result.tau):.3f}, {np.nanmax(rld_result.tau):.3f}] ns")
            
            # Create snapshot every few frames
            if frame_count % 3 == 0:
                data_manager.snapshot()
                print(f"  Created snapshot (total snapshots: {data_manager.get_frame_count() - 1})")
            
            # Stop after max_frames
            if frame_count >= max_frames:
                print(f"\nReached {max_frames} frames, stopping...")
                break
                
        elif isinstance(event, EndSeries):
            # Series ended
            print(f"\n=== Series {event.series_no} Ended ===")
            break
            
except KeyboardInterrupt:
    print("\nReceiving interrupted by user")
finally:
    receiver.stop_receiving()
    print("Receiver stopped")

print(f"\nProcessed {frame_count} frames")

## Visualizing Streaming Results

Let's visualize the processed data from the stream:

In [ ]:
if len(lifetime_results) > 0:
    # Show lifetime evolution over frames
    n_frames = min(6, len(lifetime_results))
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i in range(n_frames):
        im = axes[i].imshow(lifetime_results[i], cmap='viridis', vmin=2.0, vmax=3.5)
        axes[i].set_title(f'Frame {i+1} Lifetime', fontsize=12)
        axes[i].set_xlabel('X (pixels)')
        axes[i].set_ylabel('Y (pixels)')
        plt.colorbar(im, ax=axes[i], label='Lifetime (ns)')
    
    # Hide unused subplots
    for i in range(n_frames, 6):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("No frames were processed. Make sure the data sender is running.")

## Phasor Evolution

Visualize how the phasor distribution changes over time:

In [ ]:
if len(phasor_results) > 0:
    # Create phasor plots for multiple frames
    n_frames = min(3, len(phasor_results))
    fig, axes = plt.subplots(1, n_frames, figsize=(5*n_frames, 5))
    
    if n_frames == 1:
        axes = [axes]
    
    # Universal circle
    theta = np.linspace(0, np.pi, 100)
    circle_g = 0.5 + 0.5 * np.cos(theta)
    circle_s = 0.5 * np.sin(theta)
    
    for i in range(n_frames):
        phasor = phasor_results[i]
        g = phasor[..., 0].flatten()
        s = phasor[..., 1].flatten()
        
        # Remove NaN
        valid = ~(np.isnan(g) | np.isnan(s))
        g_valid = g[valid]
        s_valid = s[valid]
        
        # 2D histogram
        h = axes[i].hist2d(g_valid, s_valid, bins=40, cmap='viridis')
        axes[i].plot(circle_g, circle_s, 'r--', linewidth=2, label='Universal circle')
        axes[i].set_xlabel('g', fontsize=12)
        axes[i].set_ylabel('s', fontsize=12)
        axes[i].set_title(f'Frame {i+1} Phasor', fontsize=13)
        axes[i].set_aspect('equal')
        axes[i].grid(True, alpha=0.3)
        axes[i].legend(fontsize=9)
        plt.colorbar(h[3], ax=axes[i], label='Count')
    
    plt.tight_layout()
    plt.show()
else:
    print("No phasor data available.")

## Analyzing Snapshots

If we created snapshots during streaming, we can retrieve and analyze them:

In [ ]:
if data_manager is not None and data_manager.get_frame_count() > 1:
    print(f"Total frames in DataManager: {data_manager.get_frame_count()}")
    
    # Retrieve and compare snapshots
    n_snapshots = min(3, data_manager.get_frame_count())
    
    fig, axes = plt.subplots(1, n_snapshots, figsize=(5*n_snapshots, 5))
    if n_snapshots == 1:
        axes = [axes]
    
    for i in range(n_snapshots):
        # Get snapshot
        snapshot = data_manager.get_photon_count(i)
        
        # Compute intensity (sum over time)
        intensity = np.sum(snapshot, axis=-1)
        
        im = axes[i].imshow(intensity, cmap='gray')
        axes[i].set_title(f'Snapshot {i} Intensity', fontsize=13)
        axes[i].set_xlabel('X (pixels)')
        axes[i].set_ylabel('Y (pixels)')
        plt.colorbar(im, ax=axes[i], label='Total Counts')
    
    plt.tight_layout()
    plt.show()
else:
    print("No snapshots available.")

## Performance Metrics

Let's analyze the processing performance:

In [ ]:
if len(lifetime_results) > 0:
    # Estimate processing time per frame
    # (In a real streaming scenario, you would measure actual timing)
    
    frame_shape = lifetime_results[0].shape
    n_pixels = frame_shape[0] * frame_shape[1]
    
    print("Processing Statistics:")
    print(f"  Frames processed: {len(lifetime_results)}")
    print(f"  Frame size: {frame_shape[0]} x {frame_shape[1]} = {n_pixels} pixels")
    print(f"  Time bins per pixel: {time_bins}")
    print(f"  Total data points per frame: {n_pixels * time_bins:,}")
    
    # Calculate statistics
    all_lifetimes = np.concatenate([tau.flatten() for tau in lifetime_results])
    all_lifetimes = all_lifetimes[~np.isnan(all_lifetimes)]
    
    print(f"\nLifetime Statistics (all frames):")
    print(f"  Mean: {np.mean(all_lifetimes):.3f} ns")
    print(f"  Std: {np.std(all_lifetimes):.3f} ns")
    print(f"  Range: [{np.min(all_lifetimes):.3f}, {np.max(all_lifetimes):.3f}] ns")

## Delta Snapshot Mode

The library also supports delta snapshot mode, where each frame represents the difference from the previous frame. This is useful for tracking changes in dynamic samples:

In [ ]:
# Example of delta mode (conceptual - requires actual streaming data)
print("Delta Snapshot Mode:")
print("  - Each frame shows the difference from the previous frame")
print("  - Useful for tracking dynamic changes in the sample")
print("  - First frame is returned as-is (no previous frame to subtract)")
print("  - Enable by setting delta_mode=True in DataManager")

# Create a delta mode data manager
delta_manager = DataManager(
    shape=(64, 64, 256),
    dtype=np.float32,
    delta_mode=True
)

print(f"\nDelta mode DataManager created: delta_mode={delta_manager.delta_mode}")

## Summary

This notebook demonstrated streaming FLIM data processing:

1. **Stream Reception**: Set up a UDP receiver to accept FLIM data
2. **Event Handling**: Processed SeriesMetadata, ElementData, and EndSeries events
3. **Real-time Processing**: Computed phasor coordinates and lifetimes as data arrived
4. **Snapshot Management**: Created and managed snapshots during streaming
5. **Visualization**: Displayed results from multiple frames
6. **Delta Mode**: Introduced delta snapshot mode for dynamic samples

### Key Points:

- The `StreamReceiver` provides a generator interface for processing streaming data
- The `DataManager` handles frame storage with snapshot and delta mode support
- Processing can be performed in real-time as data arrives
- All results are standard NumPy arrays compatible with matplotlib

### Next Steps:

- See the selection analysis example for region-of-interest analysis
- Explore asynchronous processing for better performance
- Integrate with your TCSPC hardware for real acquisitions